In [0]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass
  

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import pandas as pd
print(tf.__version__)

2.1.0-rc1


In [0]:
TRAIN_DATA_URL = "https://drive.google.com/uc?authuser=0&id=11uwGt8fNyHm5VJbmZU6BVhJja7gjQZVQ&export=download"
TEST_DATA_URL = "https://drive.google.com/uc?authuser=0&id=1oTfHdPxVShJOpJ8wbCP0VmymyE9m5iOf&export=download"
SUB_MISSION_URL = "https://drive.google.com/a/dacon.io/uc?authuser=0&id=1jti3rkMN-Tb_WLOIeGVVFDYd_rfE9TFe&export=download"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)
submission_path = tf.keras.utils.get_file("submission_sample.csv", SUB_MISSION_URL)

In [0]:
submission = pd.read_csv(submission_path)

In [0]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [0]:
train = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)

In [0]:
train.shape, test.shape

((415423, 21), (228170, 20))

In [0]:


train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday
train['day'] = train['date'].dt.day



test['date'] = pd.to_datetime(train['date'])
test['weekday'] = test['date'].dt.weekday
test['day'] = test['date'].dt.day


date_index = pd.date_range(start='20190901', end='20191016')
date_index

DatetimeIndex(['2019-09-01', '2019-09-02', '2019-09-03', '2019-09-04',
               '2019-09-05', '2019-09-06', '2019-09-07', '2019-09-08',
               '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12',
               '2019-09-13', '2019-09-14', '2019-09-15', '2019-09-16',
               '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20',
               '2019-09-21', '2019-09-22', '2019-09-23', '2019-09-24',
               '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-28',
               '2019-09-29', '2019-09-30', '2019-10-01', '2019-10-02',
               '2019-10-03', '2019-10-04', '2019-10-05', '2019-10-06',
               '2019-10-07', '2019-10-08', '2019-10-09', '2019-10-10',
               '2019-10-11', '2019-10-12', '2019-10-13', '2019-10-14',
               '2019-10-15', '2019-10-16'],
              dtype='datetime64[ns]', freq='D')

In [0]:
date_df = pd.DataFrame(columns={'date'})
date_df['date'] = date_index
date_df.head()

,date
0,2019-09-01
1,2019-09-02
2,2019-09-03
3,2019-09-04
4,2019-09-05


In [0]:
holiday = ['2019-09-01', '2019-09-07', '2019-09-08', '2019-09-12', '2019-09-13', '2019-09-14', '2019-09-15', '2019-09-21', '2019-09-22', '2019-09-28', '2019-09-29',
          '2019-10-03', '2019-10-05', '2019-10-06', '2019-10-09','2019-10-12', '2019-10-13', '2019-10-19', '2019-10-20', '2019-10-26', '2019-10-27']
len(holiday)


21

In [0]:
holiday_df = pd.DataFrame()
holiday_df['date'] = holiday
holiday_df['date'] = pd.to_datetime(holiday_df['date'])
holiday_df.head()

,date
0,2019-09-01
1,2019-09-07
2,2019-09-08
3,2019-09-12
4,2019-09-13


In [0]:
zero = len(holiday)
holiday_df['holiday'] = np.zeros(zero)
holiday_df['holiday'] = holiday_df['holiday'].apply(int)
holiday_df.head()

,date,holiday
0,2019-09-01,0
1,2019-09-07,0
2,2019-09-08,0
3,2019-09-12,0
4,2019-09-13,0


In [0]:
holiday_df.shape

(21, 2)

In [0]:
holidays_df = pd.merge(date_df, holiday_df, how='left', on=['date'])
holidays_df = holidays_df.fillna(1) #휴일 아니면 1
holidays_df['holiday'] = holidays_df['holiday'].apply(int)
holidays_df.head()

,date,holiday
0,2019-09-01,0
1,2019-09-02,1
2,2019-09-03,1
3,2019-09-04,1
4,2019-09-05,1


In [0]:
train = pd.merge(train, holidays_df, on=['date'])
train.head()


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,weekday,day,holiday
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6,1,0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6,1,0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,6,1,0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6,1,0


In [0]:
test = pd.merge(test, holidays_df, on=['date'])
test.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,weekday,day,holiday
0,415423,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6,1,0
1,415424,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0
2,415425,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0
3,415426,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0
4,415427,2019-09-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,1,0


In [0]:
train.shape, test.shape

((415423, 24), (228170, 23))

In [0]:
train['holiday'].value_counts()


1    297247
0    118176
Name: holiday, dtype: int64

In [0]:
test['18~20_ride'] = test['11~12_ride']
test['18~20_ride'] = 0.0

test.head()


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,weekday,day,holiday,18~20_ride
0,415423,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6,1,0,0.0
1,415424,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0.0
2,415425,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0.0
3,415426,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0.0
4,415427,2019-09-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,1,0,0.0


In [0]:
#트레인

train['6~8_ride']=train['6~7_ride']+train['7~8_ride'] # 6 ~ 8시 승차인원
train['8~10_ride']=train['8~9_ride']+train['9~10_ride']
train['10~12_ride']=train['10~11_ride']+train['11~12_ride']

train['6~8_takeoff']=train['6~7_takeoff']+train['7~8_takeoff'] # 6 ~ 8시 하차인원
train['8~10_takeoff']=train['8~9_takeoff']+train['9~10_takeoff']
train['10~12_takeoff']=train['10~11_takeoff']+train['11~12_takeoff']


#테스트

test['6~8_ride']=test['6~7_ride']+test['7~8_ride'] # 6 ~ 8시 승차인원
test['8~10_ride']=test['8~9_ride']+test['9~10_ride']
test['10~12_ride']=test['10~11_ride']+test['11~12_ride']

test['6~8_takeoff']=test['6~7_takeoff']+test['7~8_takeoff'] # 6 ~ 8시 하차인원
test['8~10_takeoff']=test['8~9_takeoff']+test['9~10_takeoff']
test['10~12_takeoff']=test['10~11_takeoff']+test['11~12_takeoff']

In [0]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,weekday,day,holiday,6~8_ride,8~10_ride,10~12_ride,6~8_takeoff,8~10_takeoff,10~12_takeoff
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,1.0,7.0,8.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6,1,0,5.0,6.0,11.0,0.0,0.0,0.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6,1,0,2.0,2.0,0.0,0.0,0.0,0.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,6,1,0,17.0,32.0,30.0,0.0,0.0,0.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6,1,0,0.0,0.0,0.0,0.0,1.0,0.0


In [0]:
test.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,weekday,day,holiday,18~20_ride,6~8_ride,8~10_ride,10~12_ride,6~8_takeoff,8~10_takeoff,10~12_takeoff
0,415423,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6,1,0,0.0,8.0,9.0,10.0,0.0,0.0,1.0
1,415424,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0.0,7.0,7.0,19.0,0.0,0.0,0.0
2,415425,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0.0,6.0,4.0,3.0,0.0,0.0,0.0
3,415426,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0.0,12.0,26.0,46.0,0.0,0.0,0.0
4,415427,2019-09-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [0]:
uniq_dist_Freq_train = pd.DataFrame(train.groupby(['latitude', 'longitude']).size().rename('Freq'))
uniq_dist_Freq_test = pd.DataFrame(test.groupby(['latitude', 'longitude']).size().rename('Freq'))

In [0]:
uniq_lat_long_freq_train = uniq_dist_Freq_train.reset_index()
uniq_lat_long_freq_test = uniq_dist_Freq_test.reset_index()

In [0]:
uniq_lat_long_freq_train.shape


(3563, 3)

In [0]:
uniq_lat_long_freq_test.shape

(3504, 3)

In [0]:
DONG_DATA_URL = "https://drive.google.com/uc?authuser=0&id=19sp0k822VWiOr1tA8lhUF6qspW_eYfBi&export=download"

In [0]:
dong_file_path = tf.keras.utils.get_file("dong.xlsx", DONG_DATA_URL)

In [0]:
dong_file_path

'/root/.keras/datasets/dong.xlsx'

In [0]:
dong_cent_long_lat = pd.ExcelFile(dong_file_path).parse('합본 DB')

In [0]:
dong_cent_long_lat.columns = ["loadCode", "Province", "SiGunGu", "EupMyeonDong","Ri","lcc_latitude","lcc_longitude","CodeType"]

In [0]:
dong_cent_long_lat.head()

,loadCode,Province,SiGunGu,EupMyeonDong,Ri,lcc_latitude,lcc_longitude,CodeType
0,1111051500,서울특별시,종로구,청운효자동,NaN,37.584009,126.970626,H
1,1111053000,서울특별시,종로구,사직동,NaN,37.575408,126.965944,H
2,1111054000,서울특별시,종로구,삼청동,NaN,37.590758,126.980996,H
3,1111055000,서울특별시,종로구,부암동,NaN,37.594768,126.965574,H
4,1111056000,서울특별시,종로구,평창동,NaN,37.613029,126.974485,H


In [0]:
dong_cent_long_lat = dong_cent_long_lat[dong_cent_long_lat['Province'].str.contains('제주특별자치도', regex=False)]

In [0]:
loadcode = dong_cent_long_lat[['loadCode','lcc_latitude','lcc_longitude']]

In [0]:
loadcode['key'] = 0 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
uniq_lat_long_freq_train['key'] = 0 
uniq_lat_long_freq_test['key'] = 0 

In [0]:
uniq_lat_long_freq_train.head()

,latitude,longitude,Freq,key
0,33.20835,126.29012,42,0
1,33.20847,126.29025,44,0
2,33.21097,126.25733,322,0
3,33.21119,126.25748,308,0
4,33.21201,126.29237,13,0


In [0]:
uniq_lat_long_freq_test.head()

,latitude,longitude,Freq,key
0,33.20835,126.29012,29,0
1,33.20847,126.29025,22,0
2,33.21097,126.25733,183,0
3,33.21119,126.25748,168,0
4,33.21201,126.29237,5,0


In [0]:
full_lat_long_train = uniq_lat_long_freq_train.merge(loadcode, how='outer')
full_lat_long_test = uniq_lat_long_freq_test.merge(loadcode, how='outer')

In [0]:
full_lat_long_train.head()

,latitude,longitude,Freq,key,loadCode,lcc_latitude,lcc_longitude
0,33.20835,126.29012,42,0,5011025000,33.376347,126.284734
1,33.20835,126.29012,42,0,5011025300,33.411040,126.393958
2,33.20835,126.29012,42,0,5011025600,33.498076,126.795431
3,33.20835,126.29012,42,0,5011025900,33.472170,126.667445
4,33.20835,126.29012,42,0,5011031000,33.322497,126.223161


In [0]:
full_lat_long_test.head()

,latitude,longitude,Freq,key,loadCode,lcc_latitude,lcc_longitude
0,33.20835,126.29012,29,0,5011025000,33.376347,126.284734
1,33.20835,126.29012,29,0,5011025300,33.411040,126.393958
2,33.20835,126.29012,29,0,5011025600,33.498076,126.795431
3,33.20835,126.29012,29,0,5011025900,33.472170,126.667445
4,33.20835,126.29012,29,0,5011031000,33.322497,126.223161


In [0]:
del full_lat_long_test['key']
del full_lat_long_train['key']

In [0]:
from geopy.distance import vincenty

In [0]:
full_lat_long_train['hdist'] = full_lat_long_train.apply(lambda x: vincenty((x[0], x[1]), (x[4], x[5])).km, axis = 1)
full_lat_long_test['hdist'] = full_lat_long_test.apply(lambda x: vincenty((x[0], x[1]), (x[4], x[5])).km, axis = 1)


In [0]:
full_lat_long_train

,latitude,longitude,Freq,loadCode,lcc_latitude,lcc_longitude,hdist
0,33.20835,126.29012,42,5011025000,33.376347,126.284734,18.639244
1,33.20835,126.29012,42,5011025300,33.411040,126.393958,24.471894
2,33.20835,126.29012,42,5011025600,33.498076,126.795431,56.962683
3,33.20835,126.29012,42,5011025900,33.472170,126.667445,45.716734
4,33.20835,126.29012,42,5011031000,33.322497,126.223161,14.113685
...,...,...,...,...,...,...,...
808796,33.96364,126.29681,23,5013032022,33.348068,126.832421,84.438571
808797,33.96364,126.29681,23,5013032023,33.393144,126.794822,78.337186
808798,33.96364,126.29681,23,5013032024,33.354784,126.771072,80.594923
808799,33.96364,126.29681,23,5013032025,33.321147,126.798867,85.133021


In [0]:
full_lat_long_test

,latitude,longitude,Freq,loadCode,lcc_latitude,lcc_longitude,hdist
0,33.20835,126.29012,29,5011025000,33.376347,126.284734,18.639244
1,33.20835,126.29012,29,5011025300,33.411040,126.393958,24.471894
2,33.20835,126.29012,29,5011025600,33.498076,126.795431,56.962683
3,33.20835,126.29012,29,5011025900,33.472170,126.667445,45.716734
4,33.20835,126.29012,29,5011031000,33.322497,126.223161,14.113685
...,...,...,...,...,...,...,...
795403,33.96364,126.29681,15,5013032022,33.348068,126.832421,84.438571
795404,33.96364,126.29681,15,5013032023,33.393144,126.794822,78.337186
795405,33.96364,126.29681,15,5013032024,33.354784,126.771072,80.594923
795406,33.96364,126.29681,15,5013032025,33.321147,126.798867,85.133021


In [0]:
findloadcode_train = full_lat_long_train.loc[full_lat_long_train.groupby(['latitude', 'longitude','Freq'])["hdist"].idxmin()]
findloadcode_test = full_lat_long_test.loc[full_lat_long_test.groupby(['latitude', 'longitude','Freq'])["hdist"].idxmin()]

In [0]:
findloadcode_train

,latitude,longitude,Freq,loadCode,lcc_latitude,lcc_longitude,hdist
178,33.20835,126.29012,42,5013025021,33.223105,126.264384,2.904193
405,33.20847,126.29025,44,5013025021,33.223105,126.264384,2.906759
633,33.21097,126.25733,322,5013025022,33.221698,126.252783,1.263078
860,33.21119,126.25748,308,5013025022,33.221698,126.252783,1.244969
1123,33.21201,126.29237,13,5013031025,33.234030,126.301198,2.577123
...,...,...,...,...,...,...,...
807816,33.96269,126.29761,29,5011032021,33.963828,126.296228,0.179580
808043,33.96284,126.29564,15,5011032021,33.963828,126.296228,0.122326
808270,33.96299,126.29588,3,5011032021,33.963828,126.296228,0.098360
808497,33.96351,126.29717,2,5011032021,33.963828,126.296228,0.093937


In [0]:
findloadcode_test

,latitude,longitude,Freq,loadCode,lcc_latitude,lcc_longitude,hdist
178,33.20835,126.29012,29,5013025021,33.223105,126.264384,2.904193
405,33.20847,126.29025,22,5013025021,33.223105,126.264384,2.906759
633,33.21097,126.25733,183,5013025022,33.221698,126.252783,1.263078
860,33.21119,126.25748,168,5013025022,33.221698,126.252783,1.244969
1123,33.21201,126.29237,5,5013031025,33.234030,126.301198,2.577123
...,...,...,...,...,...,...,...
794423,33.96252,126.29429,2,5011032021,33.963828,126.296228,0.230507
794650,33.96269,126.29761,16,5011032021,33.963828,126.296228,0.179580
794877,33.96284,126.29564,5,5011032021,33.963828,126.296228,0.122326
795104,33.96299,126.29588,1,5011032021,33.963828,126.296228,0.098360


In [0]:
test_df  = pd.merge(test, findloadcode_test,  how='inner', left_on=['latitude','longitude'], right_on = ['latitude','longitude'])
train_df  = pd.merge(train, findloadcode_train,  how='inner', left_on=['latitude','longitude'], right_on = ['latitude','longitude'])

In [0]:
test_df

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,weekday,day,holiday,18~20_ride,6~8_ride,8~10_ride,10~12_ride,6~8_takeoff,8~10_takeoff,10~12_takeoff,Freq,loadCode,lcc_latitude,lcc_longitude,hdist
0,415423,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6,1,0,0.0,8.0,9.0,10.0,0.0,0.0,1.0,16,5011013700,33.486433,126.492606,0.398472
1,430688,2019-09-02,4270000,시외,344,제주썬호텔,33.48990,126.49373,2.0,5.0,3.0,1.0,8.0,15.0,0.0,0.0,0.0,0.0,0.0,1.0,0,2,1,0.0,7.0,4.0,23.0,0.0,0.0,1.0,16,5011013700,33.486433,126.492606,0.398472
2,444055,2019-09-03,4270000,시외,344,제주썬호텔,33.48990,126.49373,3.0,2.0,2.0,4.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1,0.0,5.0,6.0,11.0,0.0,0.0,0.0,16,5011013700,33.486433,126.492606,0.398472
3,455750,2019-09-03,4270000,시외,344,제주썬호텔,33.48990,126.49373,2.0,6.0,7.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1,0.0,8.0,8.0,8.0,0.0,0.0,0.0,16,5011013700,33.486433,126.492606,0.398472
4,471136,2019-09-04,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,0.0,2.0,4.0,6.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,2,4,1,0.0,0.0,6.0,9.0,1.0,0.0,0.0,16,5011013700,33.486433,126.492606,0.398472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228165,641488,2019-09-17,31520000,시내,1486,오라1동복지회관,33.49392,126.51639,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,17,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2,5011011900,33.495740,126.517606,0.231338
228166,629228,2019-09-16,20010000,시외,6115033,신양2리입구,33.94132,126.31894,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,16,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,5011032023,33.945420,126.314946,0.585793
228167,629273,2019-09-16,21010000,시내,3198,위미리(위미2리교차로),33.27737,126.66101,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,16,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,5013025327,33.280475,126.664657,0.483758
228168,636096,2019-09-17,27530000,시내,4251,꿈에그린아파트,33.28143,126.27210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,17,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,5013025030,33.280563,126.289190,1.594935


In [0]:
train_df

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,weekday,day,holiday,6~8_ride,8~10_ride,10~12_ride,6~8_takeoff,8~10_takeoff,10~12_takeoff,Freq,loadCode,lcc_latitude,lcc_longitude,hdist
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,1.0,7.0,8.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
1,11538,2019-09-02,4270000,시외,344,제주썬호텔,33.48990,126.49373,1.0,8.0,3.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,2,1,9.0,6.0,4.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
2,26498,2019-09-03,4270000,시외,344,제주썬호텔,33.48990,126.49373,2.0,4.0,2.0,7.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1,6.0,9.0,4.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
3,41983,2019-09-04,4270000,시외,344,제주썬호텔,33.48990,126.49373,3.0,3.0,1.0,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,4,1,6.0,3.0,8.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
4,56722,2019-09-05,4270000,시외,344,제주썬호텔,33.48990,126.49373,2.0,4.0,8.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3,5,1,6.0,8.0,7.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415418,374291,2019-09-28,20010000,시외,6115044,대서리,33.96252,126.29429,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,28,0,0.0,0.0,1.0,0.0,0.0,0.0,1,5011032021,33.963828,126.296228,0.230507
415419,393524,2019-09-29,27880000,시내,1368,저지리알못,33.33595,126.26332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,29,0,0.0,0.0,0.0,0.0,0.0,1.0,1,5011031030,33.331148,126.258595,0.690770
415420,397921,2019-09-29,31580000,시내,3576,어썸브리즈,33.25899,126.27554,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6,29,0,0.0,1.0,0.0,1.0,0.0,0.0,1,5013025029,33.260487,126.273607,0.244964
415421,400433,2019-09-30,22210000,시내,1585,셰프라인월드,33.43489,126.73242,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,30,1,1.0,0.0,0.0,0.0,0.0,0.0,1,5011025929,33.433063,126.676618,5.193152


In [0]:
LABEL_COLUMN = '18~20_ride'

In [0]:
train = train_df
test = test_df

In [0]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,weekday,day,holiday,6~8_ride,8~10_ride,10~12_ride,6~8_takeoff,8~10_takeoff,10~12_takeoff,Freq,loadCode,lcc_latitude,lcc_longitude,hdist
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.4899,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,0,1.0,7.0,8.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
1,11538,2019-09-02,4270000,시외,344,제주썬호텔,33.4899,126.49373,1.0,8.0,3.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,2,1,9.0,6.0,4.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
2,26498,2019-09-03,4270000,시외,344,제주썬호텔,33.4899,126.49373,2.0,4.0,2.0,7.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1,6.0,9.0,4.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
3,41983,2019-09-04,4270000,시외,344,제주썬호텔,33.4899,126.49373,3.0,3.0,1.0,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,4,1,6.0,3.0,8.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472
4,56722,2019-09-05,4270000,시외,344,제주썬호텔,33.4899,126.49373,2.0,4.0,8.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3,5,1,6.0,8.0,7.0,0.0,0.0,0.0,30,5011013700,33.486433,126.492606,0.398472


In [0]:
train.to_csv("train_feature1.csv", index=False)
test.to_csv("test_feature1.csv", index=False)
train_file_path = "train_feature1.csv"
test_file_path = "test_feature1.csv"

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=4000, # Artificially small to make examples easier to show.
      label_name= LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

SELECT_COLUMNS = ['in_out',
                  'latitude', 'longitude', 
                  '6~8_ride', '8~10_ride', '10~12_ride',
                 '18~20_ride',
                  '6~8_takeoff', '8~10_takeoff', '10~12_takeoff',
                  'weekday',	'day',	'holiday',
                  'Freq','loadCode', 'lcc_latitude',	'lcc_longitude',	'hdist'
                  ]

# SELECT_COLUMNS = ['in_out',
#                   'latitude', 'longitude', 
#                   '6~7_ride', '7~8_ride', '8~9_ride',
#                   '9~10_ride', '10~11_ride', '11~12_ride',
#                   '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
#                   '9~10_takeoff', '10~11_takeoff', '11~12_takeoff'
#                   ]


raw_train_data = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_file_path,
                           select_columns=SELECT_COLUMNS)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [0]:
show_batch(raw_test_data)

in_out              : [b'\xec\x8b\x9c\xeb\x82\xb4' b'\xec\x8b\x9c\xeb\x82\xb4'
 b'\xec\x8b\x9c\xeb\x82\xb4' ... b'\xec\x8b\x9c\xeb\x82\xb4'
 b'\xec\x8b\x9c\xeb\x82\xb4' b'\xec\x8b\x9c\xeb\x82\xb4']
latitude            : [33.234 33.491 33.49  ... 33.552 33.5   33.491]
longitude           : [126.49  126.497 126.486 ... 126.791 126.515 126.497]
weekday             : [1 2 3 ... 3 1 6]
day                 : [ 3 11 12 ...  5 10  1]
holiday             : [1 1 0 ... 1 1 0]
6~8_ride            : [0. 5. 0. ... 0. 4. 1.]
8~10_ride           : [ 1.  9. 10. ...  0. 10.  0.]
10~12_ride          : [ 0.  1.  6. ...  0. 11.  0.]
6~8_takeoff         : [0. 1. 1. ... 1. 4. 4.]
8~10_takeoff        : [ 0.  3. 19. ...  2.  6.  0.]
10~12_takeoff       : [ 0.  3.  5. ...  3. 13.  0.]
Freq                : [ 44 735 843 ...  71 511 735]
loadCode            : [5013011600 5011013700 5011013700 ... 5011025626 5011011900 5011013700]
lcc_latitude        : [33.248 33.486 33.486 ... 33.555 33.496 33.486]
lcc_longitude 

In [0]:
# SELECT_COLUMNS = ['latitude', 'longitude', 
#                   '6~7_ride', '7~8_ride', '8~9_ride',
#                   '9~10_ride', '10~11_ride', '11~12_ride',
#                   '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
#                   '9~10_takeoff', '10~11_takeoff', '11~12_takeoff'
#                   ]

DEFAULTS = [0.0,
            0.0, 0.0,
            0.0, 0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 
            0.0, 0.0, 0.0,
            0.0, 0.0, 0.0, 0.0, 0.0
            ]


temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

In [0]:
len(SELECT_COLUMNS), len(DEFAULTS)

(18, 18)

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float64) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
# NUMERIC_FEATURES = ['latitude', 'longitude', 
#                   '6~7_ride', '7~8_ride', '8~9_ride',
#                   '9~10_ride', '10~11_ride', '11~12_ride',
#                   '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
#                   '9~10_takeoff', '10~11_takeoff'
#                   ]

NUMERIC_FEATURES = [
                  'latitude', 'longitude', 
                  '6~8_ride', '8~10_ride', '10~12_ride',
                  '6~8_takeoff', '8~10_takeoff', '10~12_takeoff',
                  'day',	'holiday',
                  'Freq', 'lcc_latitude', 'lcc_longitude', 'hdist'
                  ]


packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
show_batch(raw_train_data)

in_out              : [b'\xec\x8b\x9c\xec\x99\xb8' b'\xec\x8b\x9c\xeb\x82\xb4'
 b'\xec\x8b\x9c\xeb\x82\xb4' ... b'\xec\x8b\x9c\xeb\x82\xb4'
 b'\xec\x8b\x9c\xeb\x82\xb4' b'\xec\x8b\x9c\xeb\x82\xb4']
latitude            : [33.489 33.24  33.506 ... 33.491 33.259 33.491]
longitude           : [126.485 126.564 126.493 ... 126.496 126.405 126.497]
weekday             : [5 0 4 ... 6 4 6]
day                 : [ 7 23  6 ... 15  6  1]
holiday             : [0 1 1 ... 0 1 0]
6~8_ride            : [4. 0. 0. ... 0. 1. 0.]
8~10_ride           : [3. 0. 0. ... 0. 3. 1.]
10~12_ride          : [13.  1.  0. ...  2.  5.  2.]
6~8_takeoff         : [0. 0. 0. ... 1. 0. 0.]
8~10_takeoff        : [0. 0. 4. ... 2. 0. 2.]
10~12_takeoff       : [0. 0. 5. ... 3. 0. 0.]
Freq                : [1515   31  792 ... 1334  217 1379]
loadCode            : [5011013700 5013051000 5011058000 ... 5011013700 5013062000 5011013700]
lcc_latitude        : [33.486 33.245 33.507 ... 33.486 33.254 33.486]
lcc_longitude       : [126

In [0]:
show_batch(packed_train_data)

in_out              : [b'\xec\x8b\x9c\xeb\x82\xb4' b'\xec\x8b\x9c\xec\x99\xb8'
 b'\xec\x8b\x9c\xeb\x82\xb4' ... b'\xec\x8b\x9c\xec\x99\xb8'
 b'\xec\x8b\x9c\xeb\x82\xb4' b'\xec\x8b\x9c\xeb\x82\xb4']
weekday             : [2 1 3 ... 6 1 0]
loadCode            : [5013062000 5013060000 5013011600 ... 5011013700 5011013700 5011012200]
numeric             : [[ 33.259 126.404   0.    ...  33.254 126.398   0.775]
 [ 33.236 126.478   4.    ...  33.251 126.478   1.637]
 [ 33.235 126.491   0.    ...  33.248 126.491   1.52 ]
 ...
 [ 33.489 126.485   1.    ...  33.486 126.493   0.775]
 [ 33.49  126.486   7.    ...  33.486 126.493   0.709]
 [ 33.482 126.482   1.    ...  33.482 126.476   0.562]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [0]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [0]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(14,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f56fe3f70d0>, mean=array([ 33.42 , 126.534,   1.136,   1.458,   1.179,   0.458,   0.947,
         0.811,  15.635,   0.716, 399.895,  33.421, 126.534,   0.671]), std=array([  0.108,   0.141,   2.863,   3.716,   3.362,   1.533,   2.673,
         2.509,   8.758,   0.451, 359.767,   0.108,   0.141,   0.558])))

In [0]:
example_batch['numeric']

<tf.Tensor: shape=(4000, 14), dtype=float64, numpy=
array([[ 33.489, 126.485,   0.   , ...,  33.486, 126.493,   0.775],
       [ 33.256, 126.413,   0.   , ...,  33.263, 126.413,   0.781],
       [ 33.49 , 126.486,   1.   , ...,  33.486, 126.493,   0.709],
       ...,
       [ 33.482, 126.474,   0.   , ...,  33.482, 126.476,   0.265],
       [ 33.506, 126.493,   0.   , ...,  33.507, 126.501,   0.694],
       [ 33.491, 126.496,   0.   , ...,  33.486, 126.493,   0.63 ]])>

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.64 , -0.344, -0.397, ...,  0.606, -0.291,  0.186],
       [-1.523, -0.858, -0.397, ..., -1.467, -0.856,  0.196],
       [ 0.641, -0.337, -0.047, ...,  0.606, -0.291,  0.068],
       ...,
       [ 0.569, -0.426, -0.397, ...,  0.564, -0.407, -0.729],
       [ 0.79 , -0.286, -0.397, ...,  0.794, -0.234,  0.041],
       [ 0.655, -0.263, -0.397, ...,  0.606, -0.291, -0.074]],
      dtype=float32)

In [0]:
train_loadCode = train['loadCode'].drop_duplicates().values.tolist()


In [0]:
train_loadCode

[5011013700,
 5011012200,
 5011058000,
 5013011900,
 5013062000,
 5013011400,
 5013031024,
 5013060000,
 5013051000,
 5013054000,
 5013031028,
 5013011500,
 5013059000,
 5013011600,
 5013011200,
 5013010200,
 5011011900,
 5013010300,
 5013011700,
 5013011800,
 5011032021,
 5011032022,
 5011032025,
 5011032024,
 5011032000,
 5011033000,
 5011033021,
 5011025924,
 5011061000,
 5011025921,
 5011053000,
 5011052000,
 5011025923,
 5011060000,
 5011025628,
 5011025626,
 5011025634,
 5013032021,
 5013025924,
 5013053000,
 5013010500,
 5013025321,
 5011055000,
 5011025925,
 5013025927,
 5011067000,
 5011025340,
 5011012800,
 5011025028,
 5011025333,
 5011025024,
 5013025022,
 5011025334,
 5011031028,
 5011031023,
 5011025341,
 5013025032,
 5013031021,
 5011025625,
 5013025921,
 5013025925,
 5011011700,
 5011010400,
 5011054000,
 5011011800,
 5011025929,
 5011011600,
 5011025900,
 5013032023,
 5013025324,
 5013025325,
 5013025030,
 5011025321,
 5013025024,
 5013025021,
 5013025027,
 5013031025,

In [0]:
CATEGORIES = {
    'in_out': ['시내', '시외'],
    'loadCode' : train_loadCode
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='in_out', vocabulary_list=('시내', '시외'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='loadCode', vocabulary_list=(5011013700, 5011012200, 5011058000, 5013011900, 5013062000, 5013011400, 5013031024, 5013060000, 5013051000, 5013054000, 5013031028, 5013011500, 5013059000, 5013011600, 5013011200, 5013010200, 5011011900, 5013010300, 5013011700, 5013011800, 5011032021, 5011032022, 5011032025, 5011032024, 5011032000, 5011033000, 5011033021, 5011025924, 5011061000, 5011025921, 5011053000, 5011052000, 5011025923, 5011060000, 5011025628, 5011025626, 5011025634, 5013032021, 5013025924, 5013053000, 5013010500, 5013025321, 5011055000, 5011025925, 5013025927, 5011067000, 5011025340, 5011012800, 5011025028, 5011025333, 5011025024, 5013025022, 5011025334, 5011031028, 5011031023, 5011025341, 5013025032, 5013031021, 5011025625, 5013025921, 5013025925,

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [0]:
print(preprocessing_layer(example_batch).numpy()[0])



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[ 0.     1.     1.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.

In [0]:
# rmse를 metrics로 설정하고 싶은 경우 선택합니다.
def rmse(targets, predictions):
  rmse = tf.sqrt(tf.reduce_mean(tf.math.squared_difference(targets, predictions)))
  return rmse

In [0]:

# model = tf.keras.Sequential([
#   preprocessing_layer,
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(1, activation='sigmoid'),

# ])

# model.compile(
#     loss= 'mse',
#     optimizer='adam',
#     metrics=['mean_squared_error'])


batch = 4000

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(batch, activation='relu'),
  tf.keras.layers.Dense(batch, activation='relu'),
  tf.keras.layers.Dropout(rate=0.1),
  tf.keras.layers.Dense(batch*2, activation='relu'),
  tf.keras.layers.Dense(batch*2, activation='relu'),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(batch*4, activation='elu'),
  tf.keras.layers.Dense(batch*4, activation='elu'),
  tf.keras.layers.Dropout(rate=0.1),  
  tf.keras.layers.Dense(batch*2, activation='relu'),
  tf.keras.layers.Dense(batch*2, activation='relu'),
  tf.keras.layers.Dense(batch, activation='relu'),
  tf.keras.layers.Dense(batch, activation='relu'),    
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss= 'mse',
    optimizer='adam',
    metrics=['mean_squared_error'])




In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [0]:
model.fit(train_data, steps_per_epoch=1000, epochs=30, )

Train for 1000 steps
Epoch 1/30
 104/1000 [==>...........................] - ETA: 33:44 - loss: 22.3504 - mean_squared_error: 22.3770WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 30000 batches). You may need to use the repeat() function when building your dataset.


In [0]:
# print(1.8281**(1/2)) #RMSE = root(mean_squared_error)

print(9.5848**(1/2)) #RMSE = root(mean_squared_error)


3.095932815808508


In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

     36/Unknown - 26s 731ms/step - loss: 1.0000 - mean_squared_error: 1.0000

KeyboardInterrupt: ignored

In [0]:
# print(2.0686**(1/2))


test_loss, test_accuracy = model.evaluate(test_data, verbose=1, steps=1000)
# 0.3537

In [0]:
predictions = model.predict(test_data)

In [0]:
submission['18~20_ride'] = predictions
submission.to_csv("submissionnnnn.csv", index=False)